## Cargar Dataframes


In [2]:
from pyspark.sql import SparkSession

# Inicializa SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/29 23:58:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
hdfs_path_maestra = "hdfs://namenode:9000/user/nifi/maestra.csv"
df_maestra = spark.read.csv(hdfs_path_maestra, header=True, inferSchema=True, sep=";")
df_maestra.show(truncate=False)

+-----------+------------------+-------------------------------------------------+
|Producto   |Numero de articulo|Descripcion                                      |
+-----------+------------------+-------------------------------------------------+
|LAGRICEL PF|41582             |LAGRICEL OFTENO LIBRE DE CONSERVADORES (PF) 10 ML|
|ELIPTIC PF |41561             |ELIPTIC OFTENO 5ML PF PERU                       |
|LAGRICEL   |40515             |LAGRICEL OFTENO 0.5 ML                           |
|FLUMETOL NF|40513             |FLUMETOL NF OFTENO 5ML                           |
|TRAZIDEX O |40341             |TRAZIDEX OFTENO 5 ML.                            |
|TRAZIDEX U |40342             |TRAZIDEX UNGENA 3.5 G                            |
|SOPHIPREN  |40338             |SOPHIPREN OFTENO 5 ML                            |
|GAAP       |40498             |GAAP OFTENO 3 ML                                 |
|AQUADRAN   |41945             |AQUADRAN 10G                                     |
|GAA

In [4]:
# Segundo CSV
hdfs_path_zona = "hdfs://namenode:9000/user/nifi/bd_zona.csv"
df_zona = spark.read.csv(hdfs_path_zona, header=True, inferSchema=True, sep=";")
df_zona.show(truncate=False)

+-----------+-----------------------------------------------------------------+-----------+-------+---+-------+--------+
|Vendedor   |Nombre Cliente                                                   |Producto   |MES NUM|Mes|2025   |CANTIDAD|
+-----------+-----------------------------------------------------------------+-----------+-------+---+-------+--------+
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |AGGLAD     |1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |FLUMETOL NF|1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |GAAP       |1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |LAGRICEL   |1      |ENE|0      |0       |
|Pharma - N1|BENEL PEREZ,DENNY JAVIER                                         |FLUMETOL NF|1      |ENE|898.37 |20      |
|Pharma - N1|BENEL PEREZ,DENNY J

## Manejo de datos nulos


In [5]:
from pyspark.sql.functions import col

# Contar nulos en df_maestra
df_maestra.select([
    (col(c).isNull().cast("int")).alias(c) for c in df_maestra.columns
]).groupBy().sum().show()

# Contar nulos en df_zona
df_zona.select([
    (col(c).isNull().cast("int")).alias(c) for c in df_zona.columns
]).groupBy().sum().show()


+-------------+-----------------------+----------------+
|sum(Producto)|sum(Numero de articulo)|sum(Descripcion)|
+-------------+-----------------------+----------------+
|            0|                      0|               0|
+-------------+-----------------------+----------------+

+-------------+-------------------+-------------+------------+--------+---------+-------------+
|sum(Vendedor)|sum(Nombre Cliente)|sum(Producto)|sum(MES NUM)|sum(Mes)|sum(2025)|sum(CANTIDAD)|
+-------------+-------------------+-------------+------------+--------+---------+-------------+
|            0|                  0|            0|           0|       0|        0|            0|
+-------------+-------------------+-------------+------------+--------+---------+-------------+



In [6]:
# Eliminar filas con valores nulos en maestra
df_maestra_clean = df_maestra.na.drop()

# Eliminar filas con valores nulos en zona
df_zona_clean = df_zona.na.drop()


In [7]:
# Rellenar con texto en columnas de tipo string
df_maestra_filled = df_maestra.na.fill("Desconocido")

# Rellenar con 0 en columnas numéricas de df_zona
df_zona_filled = df_zona.na.fill(0)


## Correción de tipo de datos


In [9]:
df_maestra = df_maestra.withColumn("Numero de articulo", col("Numero de articulo").cast("int"))

In [10]:
df_zona = df_zona.withColumn("2025", col("2025").cast("double")) 

In [16]:
# Filtrado de datos irrelevantes

In [17]:
df_maestra = df_maestra.filter(col("Descripcion").isNotNull())

# Filtrar zonas con ventas registradas en cero
df_zona = df_zona.filter(col("2025") > 0)


## Union


In [19]:
from pyspark.sql.functions import col

# Unión entre maestra y zona usando Producto
df_final = df_maestra.join(
    df_zona,
    df_maestra["Producto"] == df_zona["Producto"],
    how="inner"
).drop(df_zona["Producto"])  # elimina la columna duplicada

In [20]:
df_final.show()

+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+
|   Producto|Numero de articulo|         Descripcion|   Vendedor|      Nombre Cliente|MES NUM|Mes|   2025|CANTIDAD|
+-----------+------------------+--------------------+-----------+--------------------+-------+---+-------+--------+
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|Pharma - N1|BENEL PEREZ,DENNY...|      1|ENE| 898.37|      20|
| TRAZIDEX U|             40342|TRAZIDEX UNGENA 3...|Pharma - N1|BENEL PEREZ,DENNY...|      1|ENE|1028.61|      20|
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|Pharma - N1|BM CLINICA DE OJO...|      1|ENE| 612.53|      15|
|       GAAP|             40498|    GAAP OFTENO 3 ML|Pharma - N1|BM CLINICA DE OJO...|      1|ENE|1925.85|      25|
| TRAZIDEX O|             40341|TRAZIDEX OFTENO 5...|Pharma - N1|BM CLINICA DE OJO...|      1|ENE| 250.05|       6|
| TRAZIDEX U|             40342|TRAZIDEX UNGENA 3...|Pharma - N1|BM CLIN

## Feature engeeniring

In [22]:
from pyspark.sql.functions import col, count, avg, sum, max, min, when, datediff, current_date


In [23]:
df_final = df_final.withColumn("Precio_Unitario", 
                               when(col("CANTIDAD") > 0, col("2025") / col("CANTIDAD")).otherwise(0))

In [24]:
# 2. Gasto total por cliente
gasto_cliente = df_final.groupBy("Nombre Cliente").agg(sum("2025").alias("Gasto_Total"))

In [25]:
# 3. Cantidad total por cliente
cantidad_cliente = df_final.groupBy("Nombre Cliente").agg(sum("CANTIDAD").alias("Cantidad_Total"))

In [26]:
# 4. Promedio de gasto y cantidad por cliente
promedios_cliente = df_final.groupBy("Nombre Cliente").agg(
    avg("2025").alias("Gasto_Promedio"),
    avg("CANTIDAD").alias("Cantidad_Promedio")
)

# 5. Frecuencia de compra por cliente (número de transacciones)
frecuencia_cliente = df_final.groupBy("Nombre Cliente").agg(
    count("Producto").alias("Frecuencia_Compra")
)

# 6. Última compra (para calcular recencia)
ultima_compra = df_final.groupBy("Nombre Cliente").agg(
    max("MES NUM").alias("Ultimo_Mes")
)

In [27]:
# Combinar todas las features
features = gasto_cliente \
    .join(cantidad_cliente, "Nombre Cliente") \
    .join(promedios_cliente, "Nombre Cliente") \
    .join(frecuencia_cliente, "Nombre Cliente") \
    .join(ultima_compra, "Nombre Cliente")

# Mostrar resultado
features.show(10, truncate=False)

+-----------------------------------------+------------------+--------------+------------------+------------------+-----------------+----------+
|Nombre Cliente                           |Gasto_Total       |Cantidad_Total|Gasto_Promedio    |Cantidad_Promedio |Frecuencia_Compra|Ultimo_Mes|
+-----------------------------------------+------------------+--------------+------------------+------------------+-----------------+----------+
|CLINICA SAN FELIPE S A                   |21985.35          |700.0         |1099.2675         |35.0              |20               |7         |
|GOOD VISION E.I.R.L.                     |572.59            |12.0          |572.59            |12.0              |1                |3         |
|FONDO DE EMPLEADOS DEL BANCO DE LA NACION|8872.98           |229.0         |2957.66           |76.33333333333333 |3                |3         |
|CONSORCIO QORITAQQUE B Y T S.R.L.        |1427.8200000000002|29.0          |475.94000000000005|9.666666666666666 |3              

In [ ]:
##############################